### IMPORTS

In [1]:
import json
import hashlib
import pkg_resources.py2_warn

In [2]:
from web3 import Web3

In [3]:
import subprocess
import sys
import time

### LOGGING OUTLINE

In [4]:
def log(msg):
    print('[LAUNCHER] ' + msg)

### LOAD DATA FROM EXTERNAL FILES

In [5]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [6]:
with open('config/identifier.json') as json_file:
    device_info = json.load(json_file)

In [7]:
with open('config/latest.json') as json_file:
    latest = json.load(json_file)

In [8]:
log('FETCHED DATA FROM EXTERNAL FILES')

[LAUNCHER] FETCHED DATA FROM EXTERNAL FILES


### IPFS & BLOCKCHAIN GATEWAY CONNECTIONS

In [9]:
class gateway:
    def __init__(self, foo):
        self.host = foo['host']
        self.port = foo['port']

In [10]:
blockchain = gateway(settings['gateways']['blockchain'])

In [11]:
log('GATEWAY SET')

[LAUNCHER] GATEWAY SET


### BLOCKCHAIN PUBLIC/PRIVATE KEYS

In [12]:
class keypair:
    def __init__(self, foo):
        self.public = foo['public']
        self.private = foo['private']

In [13]:
keys = keypair(settings['keys'])

In [14]:
log('KEYPAIRS SET')

[LAUNCHER] KEYPAIRS SET


### HASH DEVICE JSON FOR IDENTIFIER

In [15]:
def hash_id(data):
    
    # REMOVE WHITESPACES
    to_string = json.dumps(data, sort_keys=False, indent=2)
    
    # ENCODE THE STRING WITH UTF8
    encoded = to_string.encode("utf-8")
    
    # HASH ENCODED DATA
    hashed = hashlib.sha256(encoded).hexdigest()
    
    return hashed

### DEVICE OUTLINE

In [16]:
class create_device():
    def __init__(self):
        self.name = device_info['Name']
        self.hash = hash_id(device_info)
    
    # LOCATE & SET DEVICE CONTRACT ADDRESS
    def set_contract(self, _contract):
        self.contract = _contract
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def read(self, details):
        return self.contract.read(details)
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def write(self, details):
        return self.contract.write(details)
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def event(self, name):
        return self.contract.event(name)

In [17]:
device = create_device()

In [18]:
log('DEVICE ID HASHED')

[LAUNCHER] DEVICE ID HASHED


### CONNECT TO CHAIN VIA WEBSOCKET

In [19]:
web3 = Web3(Web3.WebsocketProvider('ws://' + blockchain.host + ':' + blockchain.port))

In [20]:
if web3.isConnected():
    log('GATEWAY CONNECTION ESTABLISHED')
else:
    log('COULD NOT CONNECT TO GATEWAY. SHUTTING DOWN.')
    exit

[LAUNCHER] GATEWAY CONNECTION ESTABLISHED


### CONSTRUCT SMART CONTRACT INTERFACE

In [21]:
class contract:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': keys.public,
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = web3.eth.estimateGas(tx)
            tx['gasPrice'] = web3.toWei(20, 'gwei')
            tx['nonce'] = web3.eth.getTransactionCount(keys.public)

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = web3.eth.account.sign_transaction(tx,
                private_key = keys.private
            )

            # SEND THE TRANSACTION
            web3.eth.sendRawTransaction(signed.rawTransaction)

            # SUCCESS
            return True
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error
    
    # EVENT FILTER
    def event(self, name):
        return self.contract.events[name].createFilter(fromBlock="latest")

### STITCH TOGETHER MANAGER CONTRACTS

In [22]:
device_manager = contract(latest['devicemanager'])

In [23]:
task_manager = contract(latest['taskmanager'])

In [24]:
log('MANAGER CONTRACTS SERIALIZED')

[LAUNCHER] MANAGER CONTRACTS SERIALIZED


### CONTRUCT DEVICE CONTRACT INTERFACE

In [25]:
temp_contract = contract({
    'address': device_manager.read({
        'func': 'fetch_device',
        'params': device.hash
    }),
    'abi': latest['device']['abi']
})

In [26]:
device.set_contract(temp_contract)

In [27]:
log('DEVICE CONTRACT SERIALIZED')

[LAUNCHER] DEVICE CONTRACT SERIALIZED


### HELPER FUNCS

In [28]:
def filter_backlog(data):
    
    # FILTER ZEROS
    filtered = filter(lambda x: x != '0x0000000000000000000000000000000000000000', data)
    
    # CONVERT TO LIST & RETURN
    return list(filtered)

### TASK BACKLOG PARAM

In [29]:
raw = device.read('details')[1]

In [30]:
backlog = filter_backlog(raw)

In [31]:
log('TASKS IN BACKLOG: ' + str(len(backlog)))

[LAUNCHER] TASKS IN BACKLOG: 3


### ACTIVE STATUS PARAM

In [32]:
active = device.read('active')

In [33]:
log('CURRENT ACTIVE STATUS: ' + str(active))

[LAUNCHER] CURRENT ACTIVE STATUS: True


### EVENT FUNCTIONS

In [42]:
def update_details(event):
    
    # FETCH NEW & OLD ACTIVE STATUS
    latest = event['args']['active']
    global active

    # IF THEY DONT MATCH
    if (latest != active):
        
        # UPDATE ACTIVE STATUS
        active = latest
        
        # SEND MSG
        log('ACTIVE STATUS CHANGED TO: ' + str(latest))
        
    # UPDATE BACKLOG
    raw_backlog = event['args']['backlog']
    backlog = filter_backlog(raw_backlog)

In [35]:
def update_middleware():
    
    # PRINT REACTION
    log('MIDDLEWARE UPDATE TRIGGERED')

    # TRIGGER UPDATE SCRIPT
    # subprocess.call('./patcher')

    # CLOSE LANCHER
    # sys.exit(0)

In [36]:
def perform_task(task):
    
    # SHOW MSG
    log('STARTING TASK: ' + task)

    # TASK RETURN PARAMS
    ipfs = 'QmWATWQ7fVPP2EFGu71UkfnqhYXDYH566qy47CnJDgvs8u'
    key = '0x4f7a87EE7A53ae8606e80FE96a47038DF8ab7956'

    # SLEEP FOR 5 SECONDS
    time.sleep(5)

    # SUBMIT THE TASK RESULT
    task_manager.write({
        'func': 'complete',
        'params': [task, ipfs, key]
    })

    # SHOW MSG
    log('TASK COMPLETED')

### START LISTENING FOR EVENTS

In [37]:
log('AWAITING EVENTS...\n')

[LAUNCHER] AWAITING EVENTS...



### CONTRACT EVENTS

In [38]:
update_event = device.event('middleware')

In [39]:
changes_event = device.event('changes')

In [40]:
assignment_event = device.event('assignment')

### EVENT LOOP

In [41]:
while(True):
    
    # ACTIVE STATUS EVENT
    for event in changes_event.get_new_entries():
        update_details(event)
    
    # UPDATE MIDDLEWARE EVENT
    for event in update_event.get_new_entries():
        update_middleware()
        
    # PERFORM TASK IN BACKLOG
    for task in backlog:
        perform_task(task)

[LAUNCHER] STARTING TASK: 0x67a0f267E051A6acC77E3C1ea3c690147991a8ea
[LAUNCHER] TASK COMPLETED
[LAUNCHER] STARTING TASK: 0x40cB52a87042548b5237b0bDeC648826EB3D7f87
[LAUNCHER] TASK COMPLETED
[LAUNCHER] STARTING TASK: 0xba0A77FBD93e46a1a4A3F8ffC9fE13A776a14D5e
[LAUNCHER] TASK COMPLETED
AttributeDict({'args': AttributeDict({'owner': '0x4f7a87EE7A53ae8606e80FE96a47038DF8ab7956', 'backlog': ['0x0000000000000000000000000000000000000000', '0x0000000000000000000000000000000000000000', '0x0000000000000000000000000000000000000000', '0x0000000000000000000000000000000000000000', '0x0000000000000000000000000000000000000000', '0x0000000000000000000000000000000000000000', '0x40cB52a87042548b5237b0bDeC648826EB3D7f87', '0xba0A77FBD93e46a1a4A3F8ffC9fE13A776a14D5e'], 'completed': 6, 'tags': 'eyJmb28iOiJiYXIiLCJleXkiOiJsbWFvIn0=', 'active': True, 'discoverable': False}), 'event': 'changes', 'logIndex': 3, 'transactionIndex': 0, 'transactionHash': HexBytes('0xc706fd0ded3eaa849b2ed71f05dafdf7fb3db66f93f6718

KeyboardInterrupt: 